## Import Dependancies

In [7]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier, KDTree
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from wordcloud import WordCloud, STOPWORDS
from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import confusion_matrix, classification_report

In [8]:
# File to Load (Remember to Change These)
data = "../Resources/FINAL_CLEAN_FILE.csv"

# read to dataframe
data_df = pd.read_csv(data)
data_df.head()

,VAERS_ID,SYMPTOM,ASSIGNED_GROUP,SEVERITY_LEVEL,DIED,L_THREAT,HOSPITAL,AGE_YRS,AGE_GROUP,OTHER_MEDS,...,VAX_SITE_RA,18-25,26-35,36-45,46-55,56-65,66-75,76-85,86-95,96 +
0,916600,Dysphagia,Difficulty swallowing,MILD,0.0,0.0,0.0,33.0,26-35,0.0,...,0.0,0,1,0,0,0,0,0,0,0
1,917168,Dysphagia,Difficulty swallowing,MILD,0.0,0.0,0.0,64.0,56-65,1.0,...,1.0,0,0,0,0,1,0,0,0,0
2,918386,Dysphagia,Difficulty swallowing,MILD,0.0,0.0,0.0,62.0,56-65,1.0,...,1.0,0,0,0,0,1,0,0,0,0
3,918393,Dysphagia,Difficulty swallowing,MILD,0.0,0.0,0.0,62.0,56-65,1.0,...,1.0,0,0,0,0,1,0,0,0,0
4,918479,Dysphagia,Difficulty swallowing,MILD,0.0,0.0,0.0,37.0,36-45,1.0,...,1.0,0,0,1,0,0,0,0,0,0


In [9]:
severity = data_df['SEVERITY_LEVEL'].unique()

In [63]:
data_df.groupby(['SEVERITY_LEVEL']).count()

,VAERS_ID,SYMPTOM,ASSIGNED_GROUP,DIED,L_THREAT,HOSPITAL,AGE_YRS,AGE_GROUP,OTHER_MEDS,CUR_ILL,...,VAX_SITE_RA,18-25,26-35,36-45,46-55,56-65,66-75,76-85,86-95,96 +
SEVERITY_LEVEL,,,,,,,,,,,,,,,,,,,,,
MILD,23836,23836,23836,23836,23836,23836,23836,23836,23836,23836,...,23836,23836,23836,23836,23836,23836,23836,23836,23836,23836
MODERATE,1984,1984,1984,1984,1984,1984,1984,1984,1984,1984,...,1984,1984,1984,1984,1984,1984,1984,1984,1984,1984
SEVERE,662,662,662,662,662,662,662,662,662,662,...,662,662,662,662,662,662,662,662,662,662


## Prep Data For Model

In [43]:
# create train data
X = data_df.drop(['VAX_SITE_LA', 'VAX_SITE_RA','DIED','L_THREAT','VAERS_ID', 'SYMPTOM','ASSIGNED_GROUP','SEVERITY_LEVEL','HOSPITAL','AGE_YRS', 'AGE_GROUP'], axis=1)
X

,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,ALLERGIES,F,M,JANSSEN,MODERNA,PFIZER,...,VAX_DOSE_SERIES_2,18-25,26-35,36-45,46-55,56-65,66-75,76-85,86-95,96 +
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0,1,0,0,0,0,0,0,0
1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0,0,0,0,1,0,0,0,0
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0,0,0,0,1,0,0,0,0
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0,0,0,0,1,0,0,0,0
4,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26477,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0,0,0,0,0,1,0,0,0
26478,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0,0,0,0,0,0,1,0,0
26479,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0,0,0,0,0,0,1,0,0
26480,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0,0,0,0,0,0,1,0,0


In [44]:
# convert the serverity labels to numbers and create the target data
y = LabelEncoder().fit_transform(data_df['SEVERITY_LEVEL'])
y

array([0, 0, 0, ..., 2, 1, 1])

In [45]:
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)

In [46]:
# Scaling the train data with StandardScaler()
#scaler = StandardScaler().fit(X_train)
#X_train_scaled = scaler.transform(X_train)
#X_test_scaled = scaler.transform(X_test)

## Testing KNN to Determine Best k value

In [47]:
lr_classifier = LogisticRegression(solver= 'lbfgs', max_iter=200)
lr_classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200)

In [48]:
#print the model score
print(f"Training Data Score: {lr_classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {lr_classifier.score(X_test, y_test)}")

Training Data Score: 0.9001560847892855
Testing Data Score: 0.8998640688717716


In [50]:
# Sample Prediction 1
sample_user1 = [[0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1]]
predicted_symptoms = lr_classifier.predict(sample_user1)
predicted_symptoms

array([0])

In [57]:
# Sample Prediction 2 (return severe)
sample_user3 = [[1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
predicted_symptoms = lr_classifier.predict(sample_user3)
print(severity[predicted_symptoms])

['MILD']


In [36]:
X.columns

Index(['OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'ALLERGIES', 'F', 'M',
       'JANSSEN', 'MODERNA', 'PFIZER', 'VAX_DOSE_SERIES_1',
       'VAX_DOSE_SERIES_2', 'VAX_SITE_LA', 'VAX_SITE_RA', '18-25', '26-35',
       '36-45', '46-55', '56-65', '66-75', '76-85', '86-95', '96 +'],
      dtype='object')

Based on the scores above, we will move forward with 5 neighbors for our model. This was run on 3 random states (33, 42, 7) and all began to stabilize at k=5. 

## Create KNN Model 

In [10]:
# Note that k: 5 provides the best accuracy where the classifier starts to stablize
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
print('k=5 Test Acc: %.3f' % knn.score(X_test, y_test))

k=5 Test Acc: 0.837


In [ ]:
X.columns

### Test Prediction 1

In [ ]:
# 5 people closest to user 1
tree = KDTree(X)
dist, ind = tree.query(sample_user1, k=200)

#convery ndarry to list 
like_users = ind[0].tolist()
print(like_users)  # indices of 5 closest neighbors

In [ ]:
# save nearest 100 neighbors symptoms as list for wordcloud
predicted_symptoms = data_df['ASSIGNED_GROUP'].iloc[like_users].tolist()
predicted_symptoms[:5]

In [ ]:
# save nearest 100 neighbors severity levels as list for % tables
predicted_severity = data_df['SEVERITY_LEVEL'].iloc[like_users].tolist()
predicted_severity[:5]

In [ ]:
# count the number of occurrences from nearest neighboors
def count_levels(lst):
    mld_ct = 0
    mod_ct = 0
    sev_ct = 0 
    for person in lst:
        if (person == 'MILD'):
            mld_ct = mld_ct + 1
        if (person == 'MODERATE'):
            mod_ct = mod_ct + 1
        if (person == 'SEVERE'):
            sev_ct = sev_ct + 1    
            
    return mld_ct, mod_ct, sev_ct

In [ ]:
count_levels(predicted_severity)

In [ ]:
cloud_maker(predicted_symptoms)

In [ ]:
# save nearest 50 neighbors symptoms as list for wordcloud
predicted_symptoms = data_df['ASSIGNED_GROUP'].iloc[like_users].tolist()
predicted_symptoms[:5]

In [ ]:
def cloud_maker(words):
    #convert list to string and generate
    unique_string=(" ").join(predicted_symptoms)

    #create circle mask
    x, y = np.ogrid[:300, :300]
    mask = (x - 150) ** 2 + (y - 150) ** 2 > 130 ** 2
    mask = 255 * mask.astype(int)
    
    wordcloud = WordCloud(background_color="white", repeat=True,
                width = 1000, height = 500, mask=mask).generate(unique_string)
    
    plt.figure(figsize=(15,8))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.savefig("curr_cloud.png", bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
cloud_maker(predicted_symptoms)

In [ ]:
severity

In [ ]:
#convert list to string and generate
unique_string=(" ").join(predicted_symptoms)

#create circle mask
x, y = np.ogrid[:300, :300]
mask = (x - 150) ** 2 + (y - 150) ** 2 > 130 ** 2
mask = 255 * mask.astype(int)


#test mask
from PIL import Image
from os import path
# get data directory (using getcwd() is needed to support running example in generated IPython notebook)
d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
new_mask = np.array(Image.open(path.join(d, 'circle_mask.png')))

wordcloud = WordCloud(background_color="white", repeat=True,
                      width = 1000, height = 500, mask=mask).generate(unique_string)

plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear")
#plt.savefig("your_file_name"+".png", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# Sample Prediction 2
sample_user2 = [[0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]]
predicted_symptoms = knn.predict(sample_user2)
print(severity[predicted_symptoms])

In [ ]:
predicted_severity = data_df['SEVERITY_LEVEL'].iloc[like_users].tolist()
predicted_severity[:5]

In [ ]:
count_levels(predicted_severity)

In [ ]:
tree = KDTree(X)
ind = tree.query([[1.25, 1.35]], k=5)
print(ind)  # indices of 5 closest neighbors


print('People like you reported the following symptoms:')